# 8.4 循环神经网络
- **目录**
  - 8.4.1 无隐状态的神经网络
  - 8.4.2 有隐状态的循环神经网络
  - 8.4.3 基于循环神经网络的字符级语言模型
  - 8.4.4 困惑度（Perplexity）


在8.3节中，
我们介绍了$n$元语法模型，
其中单词$x_t$在时间步$t$的条件概率仅取决于前面$n-1$个单词。
对于时间步$t-(n-1)$之前的单词，
如果我们想将其可能产生的影响合并到$x_t$上，
需要增加$n$，然而模型参数的数量也会随之呈指数增长，
因为词表$\mathcal{V}$需要存储$|\mathcal{V}|^n$个数字，
因此与其将$P(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$模型化，
不如使用**隐变量模型**：

$$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1}) \tag{8.4.1}$$

**其中$h_{t-1}$是*隐状态*（hidden state），
也称为隐藏变量（hidden variable），
它存储了到时间步$t-1$的序列信息。**
通常，我们可以基于当前输入$x_{t}$和先前隐状态$h_{t-1}$
来计算时间步$t$处的任何时间的隐状态：

$$h_t = f(x_{t}, h_{t-1}) \tag{8.4.2}$$

对于公式 8.4.2中的函数$f$，隐变量模型不是近似值。
毕竟$h_t$是可以仅仅存储到目前为止观察到的所有数据，
然而这样的操作可能会使计算和存储的代价都变得昂贵。

回想一下，我们在第4章中
讨论过的具有隐藏单元的隐藏层。
值得注意的是，**隐藏层和隐状态指的是两个截然不同的概念**。
如上所述，**隐藏层是在从输入到输出的路径上（以观测角度来理解）的隐藏的层，
而隐状态则是在给定步骤所做的任何事情（以技术角度来定义）的输入，
并且这些状态只能通过先前时间步的数据来计算**。

**循环神经网络（recurrent neural networks，RNNs）
是具有隐状态的神经网络**。
在介绍循环神经网络模型之前，
我们首先回顾4.1中介绍的多层感知机模型。

- **要点：**
  - **n元语法模型的局限性**：在n元语法模型中，单词$x_t$在时间步$t$的条件概率仅取决于前面$n-1$个单词。这导致模型参数数量随着n的增加呈指数增长。为了解决这个问题，可以使用隐变量模型。
  - **隐变量模型**：隐变量模型使用隐状态$h_{t-1}$来存储序列信息，公式为$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1})$。隐状态$h_t$可以根据当前输入和先前隐状态计算得到：$h_t = f(x_{t}, h_{t-1})$。
  - **隐状态与隐藏层的区别**：隐藏层是指在从输入到输出的路径上的隐藏层，而隐状态是指在给定步骤所做的任何事情的输入，这些状态只能通过先前时间步的数据来计算。
  - **循环神经网络（RNNs）**：循环神经网络是具有隐状态的神经网络，可以有效地处理序列数据。

## 8.4.1 无隐状态的神经网络

让我们来看一看只有单隐藏层的多层感知机。
设隐藏层的激活函数为$\phi$，
给定一个小批量样本$\mathbf{X} \in \mathbb{R}^{n \times d}$，
其中批量大小为$n$，输入维度为$d$，
则隐藏层的输出$\mathbf{H} \in \mathbb{R}^{n \times h}$通过下式计算：

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h) \tag{8.4.3}$$


在公式8.4.3中，
我们拥有的隐藏层权重参数为$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$，
偏置参数为$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$，
以及隐藏单元的数目为$h$。
因此求和时可以应用广播机制（见2.1.3节）。
接下来，将**隐藏变量$\mathbf{H}$用作输出层的输入**。
输出层由下式给出：

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q \tag{8.4.4}$$

其中，$\mathbf{O} \in \mathbb{R}^{n \times q}$是输出变量，
$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$是权重参数，
$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$是输出层的偏置参数。
如果是分类问题，我们可以用$\text{softmax}(\mathbf{O})$
来计算输出类别的概率分布。

这完全类似于之前在8.1节中解决的回归问题，
因此我们省略了细节。
无须多言，只要可以随机选择“特征-标签”对，
并且通过自动微分和随机梯度下降能够学习网络参数就可以了。

- **要点：**
  - **单隐藏层的多层感知机**：考虑一个只有单隐藏层的多层感知机，其隐藏层的激活函数为$\phi$。给定一个小批量样本$\mathbf{X} \in \mathbb{R}^{n \times d}$，其中批量大小为$n$，输入维度为$d$。
  - **隐藏层输出**：隐藏层的输出$\mathbf{H} \in \mathbb{R}^{n \times h}$可以通过公式$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h)$计算，其中$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$是隐藏层权重参数，$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$是偏置参数，$h$是隐藏单元的数目。
  - **输出层**：将隐藏变量$\mathbf{H}$用作输出层的输入，输出层由公式$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q$给出，其中$\mathbf{O} \in \mathbb{R}^{n \times q}$是输出变量，$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$是权重参数，$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$是输出层的偏置参数。
  - **分类问题**：对于分类问题，可以使用$\text{softmax}(\mathbf{O})$来计算输出类别的概率分布。
  - **参数学习**：可以通过自动微分和随机梯度下降来学习网络参数。


---
- **说明：公式 8.4.4 中的隐藏变量，并与 RNN 中的隐状态**
  - 在公式 8.4.4 中，隐藏变量 $\mathbf{H}$ 是一个只有单隐藏层的多层感知机（MLP）中隐藏层的输出。
    - 这个隐藏层是在输入层和输出层之间的一个层，用于对输入数据进行特征提取和信息处理。
    - 在这个例子中，$\mathbf{H}$ 是一个矩阵，其形状为 $(n, h)$，其中 $n$ 是批量大小，$h$ 是隐藏单元的数量。
    - 隐藏变量 $\mathbf{H}$ 是通过将输入数据 $\mathbf{X}$ 与隐藏层的权重矩阵 $\mathbf{W}_{xh}$ 相乘并加上偏置 $\mathbf{b}_h$，然后应用激活函数 $\phi$ 计算得到的。
  - 与之不同，RNN 中的隐状态是一个序列模型中的内部状态，用于在每个时间步捕捉输入序列中的**依赖关系**。
    - 在 RNN 中，隐状态在每个时间步更新并传递，以便在序列中保留上下文信息。
    - 在处理序列数据时，隐状态允许模型捕捉长距离依赖关系。
    - 与前向传播的多层感知机不同，RNN 的隐状态在每个时间步**根据当前输入和前一时间步的隐状态进行更新**。
  - 公式 8.4.4 中的隐藏变量（$\mathbf{H}$）是指 MLP 中隐藏层的输出，它在输入层和输出层之间进行特征提取和信息处理。
    - 与之相反，RNN 中的隐状态是序列模型中的内部状态，用于捕捉输入序列的依赖关系。
    - 这两个概念在功能和用途上有明显的差异：**隐藏变量主要用于特征提取，而隐状态主要用于捕捉序列依赖关系**。
---

## 8.4.2 有隐状态的循环神经网络


有了隐状态后，情况就完全不同了。
假设我们在时间步$t$有小批量输入$\mathbf{X}_t \in \mathbb{R}^{n \times d}$。
换言之，对于$n$个序列样本的小批量，
$\mathbf{X}_t$的每一行对应于来自该序列的时间步$t$处的一个样本。


接下来，<b>用$\mathbf{H}_t  \in \mathbb{R}^{n \times h}$表示时间步$t$的隐藏变量。</b>

与多层感知机不同的是，
我们在这里保存了前一个时间步的隐藏变量$\mathbf{H}_{t-1}$，
并引入了一个新的权重参数$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$，
来描述如何在当前时间步中使用前一个时间步的隐藏变量。

具体地说，**当前时间步隐藏变量由当前时间步的输入
与前一个时间步的隐藏变量一起计算得出**：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h) \tag{8.4.5}$$


与公式8.4.3 相比，
公式8.4.5多添加了一项
$\mathbf{H}_{t-1} \mathbf{W}_{hh}$，
从而实例化了公式8.4.2。

从相邻时间步的隐藏变量$\mathbf{H}_t$和
$\mathbf{H}_{t-1}$之间的关系可知，
这些变量捕获并保留了序列直到其当前时间步的<b>历史信息</b> ，
就如当前时间步下神经网络的状态或记忆，
因此这样的隐藏变量被称为**隐状态（hidden state）**。

由于在当前时间步中，
隐状态使用的定义与前一个时间步中使用的定义相同，
因此公式8.4.5的计算是**循环的（recurrent）**。

于是基于循环计算的隐状态神经网络被命名为
**循环神经网络（recurrent neural network）**。
在循环神经网络中执行公式8.4.5计算的层
称为**循环层（recurrent layer）。**

有许多不同的方法可以构建循环神经网络，
由公式8.4.5定义的隐状态的循环神经网络是非常常见的一种。

对于时间步$t$，输出层的输出类似于多层感知机中的计算：

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q \tag{8.4.6}$$

循环神经网络的参数包括隐藏层的权重
$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$和偏置$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$，
以及输出层的权重$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$
和偏置$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$。

值得一提的是，即使在不同的时间步，循环神经网络也总是使用这些模型参数。
因此，循环神经网络的参数开销不会随着时间步的增加而增加。

图8.4.1展示了循环神经网络在三个相邻时间步的计算逻辑。
在任意时间步$t$，隐状态的计算可以被视为：

1. 拼接当前时间步$t$的输入$\mathbf{X}_t$和前一时间步$t-1$的隐状态$\mathbf{H}_{t-1}$；
1. 将拼接的结果送入带有激活函数$\phi$的全连接层。
   全连接层的输出是当前时间步$t$的隐状态$\mathbf{H}_t$。
   
在本例中，模型参数是$\mathbf{W}_{xh}$和$\mathbf{W}_{hh}$的拼接，
以及$\mathbf{b}_h$的偏置，所有这些参数都来自图8.4.1。
当前时间步$t$的隐状态$\mathbf{H}_t$
将参与计算下一时间步$t+1$的隐状态$\mathbf{H}_{t+1}$。
而且$\mathbf{H}_t$还将送入全连接输出层，
用于计算当前时间步$t$的输出$\mathbf{O}_t$。

<center>
    <img src="../img/rnn.svg" alt="具有隐状态的循环神经网络">
</center>
<center>
    图8.4.1 具有隐状态的循环神经网络
</center>

我们刚才提到，**隐状态中
$\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$的计算，
相当于$\mathbf{X}_t$和$\mathbf{H}_{t-1}$的拼接
与$\mathbf{W}_{xh}$和$\mathbf{W}_{hh}$的拼接的矩阵乘法**。
虽然这个性质可以通过数学证明，
但在下面我们使用一个简单的代码来说明一下。
首先，我们定义矩阵`X`、`W_xh`、`H`和`W_hh`，
它们的形状分别为$(3，1)$、$(1，4)$、$(3，4)$和$(4，4)$。
分别将`X`乘以`W_xh`，将`H`乘以`W_hh`，
然后将这两个乘法相加，我们得到一个形状为$(3，4)$的矩阵。

- **要点：**
  - 循环神经网络（RNN）通过维护一个**隐状态来捕捉并保留序列的历史信息**。
  - 当前时间步的隐状态由当前时间步的输入和前一个时间步的隐状态共同计算得出。
  - RNN中的循环计算是通过在相邻时间步之间保留隐状态并引入新的权重参数来实现的。
  - 循环神经网络的参数不会随着时间步的增加而增加。
  - RNN在任意时间步$t$的计算逻辑包括：
    - 拼接当前时间步$t$的输入$\mathbf{X}_t$和前一时间步$t-1$的隐状态$\mathbf{H}_{t-1}$。
    - 将拼接的结果送入带有激活函数$\phi$的全连接层，输出为当前时间步$t$的隐状态$\mathbf{H}_t$。
  - 当前时间步的隐状态将参与计算下一时间步的隐状态，并送入全连接输出层以计算当前时间步的输出。

In [2]:
%matplotlib inline
import torch
from d2l import torch as d2l

In [3]:
'''
X输入的全连接与H隐状态的计算：
Ox=(3,1)@(1,4)=(3,4)
Oh=(3,4)@(4,4)=(3,4)
Ox+Oh=(3,4)
'''
X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 4.9124, -0.9015, -2.9711, -1.7230],
        [ 1.4999,  1.4585, -2.1301, -3.2863],
        [ 3.7903, -1.6917, -1.7921, -0.4254]])

- 现在，我们沿列（轴1）拼接矩阵`X`和`H`，沿行（轴0）拼接矩阵`W_xh`和`W_hh`。
- 这两个拼接分别产生形状$(3, 5)$和形状$(5, 4)$的矩阵。
- 再将这两个拼接的矩阵相乘，得到与上面相同形状$(3, 4)$的输出矩阵。


In [4]:
'''
X|H=(3,1)|(3,4)=(3,5),横向拼接
W_xh|W_hh=(1,4)--(4,4)=(5,4),纵向拼接
二者求点积：(3,4)
'''
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 4.9124, -0.9015, -2.9711, -1.7230],
        [ 1.4999,  1.4585, -2.1301, -3.2863],
        [ 3.7903, -1.6917, -1.7921, -0.4254]])

## 8.4.3 基于循环神经网络的字符级语言模型

回想一下 8.3节中的语言模型，
我们的目标是根据过去的和当前的词元预测下一个词元，
因此我们将原始序列**移位一个词元作为标签**。
Bengio等人首先提出使用神经网络进行语言建模（Bengio, Y., Ducharme, R., Vincent, P., & Jauvin, C. (2003). A neural probabilistic language model. ）。
接下来，我们看一下如何使用循环神经网络来构建语言模型。
设小批量大小为1，批量中的那个文本序列为“machine”。
为了简化后续部分的训练，我们考虑使用
**字符级语言模型（character-level language model）**，
将文本词元化为字符而不是单词。
图8.4.2演示了
如何通过基于字符级语言建模的循环神经网络，
使用当前的和先前的字符预测下一个字符。

<center>
    <img src="../img/rnn-train.svg" alt="基于循环神经网络的字符级语言模型">
</center>
<center>
    图8.4.2基于循环神经网络的字符级语言模型：输入序列和标签序列分别为“machin”和“achine”
</center>
在训练过程中，我们对<b>每个时间步</b>的输出层的输出进行softmax操作，
然后利用交叉熵损失计算模型输出和标签之间的误差。
由于隐藏层中隐状态的循环计算，图8.4.2中的第$3$个时间步的输出$\mathbf{O}_3$
由文本序列“m”、“a”和“c”确定。
由于训练数据中这个文本序列的下一个字符是“h”，
因此第$3$个时间步的损失将取决于下一个字符的概率分布，
而下一个字符是基于特征序列“m”、“a”、“c”和这个时间步的标签“h”生成的。

在实践中，我们使用的批量大小为$n>1$，
每个词元都由一个$d$维向量表示。
因此，在时间步$t$输入$\mathbf X_t$将是一个$n\times d$矩阵，
这与我们在8.4.2节中的讨论相同。

- **要点：**
  - 语言模型的目标是根据过去的和当前的词元预测下一个词元。
  - Bengio等人首次提出了使用神经网络进行语言建模。
  - 字符级语言模型将文本词元化为字符而不是单词。
  - 基于字符级语言模型的循环神经网络使用当前的和先前的字符预测下一个字符。
  - 在训练过程中，对每个时间步的输出层的输出进行softmax操作，然后利用**交叉熵损失计算模型输出和标签之间的误差**。
  - 隐藏层中隐状态的循环计算使得输出由整个文本序列的历史信息决定。
  - 在实践中，批量大小通常大于1，且每个词元由一个高维向量表示。

## 8.4.4 困惑度（Perplexity）

最后，让我们讨论如何度量语言模型的质量，
这将在后续部分中用于评估基于循环神经网络的模型。
一个好的语言模型能够用高度准确的词元来预测我们接下来会看到什么。
考虑一下由不同的语言模型给出的对“It is raining ...”（“...下雨了”）的**续写**：

1. "It is raining outside"（外面下雨了）
1. "It is raining banana tree"（香蕉树下雨了）
1. "It is raining piouw;kcj pwepoiut"（piouw;kcj pwepoiut下雨了）

就质量而言，例$1$显然是最合乎情理、在逻辑上最连贯的。
虽然这个模型可能没有很准确地反映出后续词的语义，
比如，“It is raining in San Francisco”（旧金山下雨了）
和“It is raining in winter”（冬天下雨了）
可能才是更完美的合理扩展，
但该模型已经能够捕捉到跟在后面的是哪类单词。
例$2$则要糟糕得多，因为其产生了一个**无意义的续写**。
尽管如此，至少该模型已经学会了如何拼写单词，
以及单词之间的某种程度的相关性。
最后，例$3$表明了**训练不足的模型是无法正确地拟合数据的**。

我们可以通过计算序列的**似然概率**来度量模型的**质量**。
然而这是一个难以理解、难以比较的数字。
毕竟，较短的序列比较长的序列更有可能出现，
因此评估模型产生托尔斯泰的巨著《战争与和平》的可能性
不可避免地会比产生圣埃克苏佩里的中篇小说《小王子》可能性要小得多。**而缺少的可能性值相当于平均数**。


在这里，信息论可以派上用场了。
我们在引入softmax回归
（ 3.4.7节）时定义了熵、惊异和交叉熵，
并在[信息论的在线附录](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html)
中讨论了更多的信息论知识。
如果想要压缩文本，我们可以根据当前词元集预测的下一个词元。
一个更好的语言模型应该能让我们更准确地预测下一个词元。
因此，它应该允许我们在压缩序列时花费更少的比特。
所以我们可以通过一个序列中所有的<b>$n$个词元的交叉熵损失的平均值</b>来衡量：

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1) \tag{8.4.7}$$


其中$P$由语言模型给出，
$x_t$是在时间步$t$从该序列中观察到的实际词元。
这使得不同长度的文档的性能具有了可比性。

由于历史原因，自然语言处理的科学家更喜欢使用一个叫做**困惑度（perplexity）** 的量。
简而言之，它是公式8.4.7的指数：

$${\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right)} \tag{8.4.8}$$

**困惑度的最好的理解是“下一个词元的实际选择数的调和平均数”**。
我们看看一些案例：

* 在最好的情况下，模型总是完美地估计标签词元的概率为1。
  在这种情况下，模型的困惑度为1。
* 在最坏的情况下，模型总是预测标签词元的概率为0。
  在这种情况下，困惑度是正无穷大。
* 在基线上，该模型的预测是词表的所有可用词元上的均匀分布。
  在这种情况下，困惑度等于词表中唯一词元的数量。
  事实上，如果我们在没有任何压缩的情况下存储序列，
  这将是我们能做的最好的编码方式。
  因此，这种方式提供了一个重要的上限，
  而任何实际模型都必须超越这个上限。

在接下来的小节中，我们将基于循环神经网络实现字符级语言模型，
并使用困惑度来评估这样的模型。

- **要点：**
  - 困惑度用于度量语言模型的质量，衡量模型预测下一个词元的准确性。
  - 一个好的语言模型允许更准确地预测下一个词元，并在压缩序列时使用更少的比特。
  - 可以通过计算序列中所有词元的交叉熵损失的平均值来衡量模型的质量。
  - 困惑度是交叉熵损失平均值的指数，使得不同长度的文档的性能具有可比性。
  - 困惑度可以理解为“下一个词元的实际选择数的调和平均数”。
  - 困惑度的三种情况：
    - 最好情况：模型总是完美地估计标签词元的概率为1，困惑度为1。
    - 最坏情况：模型总是预测标签词元的概率为0，困惑度是正无穷大。
    - 基线情况：模型预测是词表的所有可用词元上的均匀分布，困惑度等于词表中唯一词元的数量。

----
- **说明：**
- **（1）语言模型与交叉熵之间的关系**
  - 一般情况下，交叉熵 $ H(P, Q) $ 的形式为：
  $$
  H(P, Q) = - \sum_x P(x) \log Q(x) \tag{3.4.12}
  $$
  其中，$ P(x) $ 是真实分布，$ Q(x) $ 是模型预测的分布。
  - **概率分布的选择**：在语言模型中，真实分布 $ P $ 通常用训练数据的实际分布表示，而模型给出的条件概率 $ P(x_t \mid x_{t-1}, \ldots, x_1) $ 是模型对真实分布的估计。
  - **序列的交叉熵**：最小化预测序列与真实序列之间的差异。
    - 对于给定的序列 $ x_1, x_2, \ldots, x_n $，交叉熵可以表示为：
   $$
   H(P, Q) = - \sum_{t=1}^n P(x_t \mid x_{t-1}, \ldots, x_1) \log Q(x_t \mid x_{t-1}, \ldots, x_1)
   $$
    - 在训练数据中，训练数据中给定的序列就是真实分布，每个词的真实概率 $ P(x_t \mid x_{t-1}, \ldots, x_1) $ 可以看作是 1，
  - **平均交叉熵**：为了得到句子的平均交叉熵，将序列的总交叉熵除以序列长度 $ n $：
   $$
   H = -\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)
   $$
  - 语言模型的交叉熵实际上是计算整个句子中每个词的对数似然平均值，从而衡量模型对给定数据集的预测质量。
  - 通过最小化这个值，模型能够更好地捕获语言的统计特性。
  - 8.5节中的循环神经网络实现使用的就是交叉熵损失。


- **（2）为何说平均交叉熵损失使得不同长度的文档具有可比性？**
  - **长度归一化**：通过将总的损失除以序列的长度 $n$，我们计算的是**每个词元**的平均损失，而不是整个序列的总损失。
    - 这样就避免了较长序列的损失值自然增大，从而使得不同长度的文本可以进行公平的比较。
  - **每个词元的预测难度相似**：每个词元的损失值反映了模型对于该词元的预测准确性，而与序列的总长度无关。
    - 因此，对于不同长度的序列，平均交叉熵损失能够有效衡量模型在每个词元上的预测质量，而不会因为序列的长短而偏向某一类序列。
  - **信息压缩的视角**：从信息论的角度来看，交叉熵损失可以解释为针对每个词元的压缩代价，衡量模型对每个词元的预测信息量。
    - 如果模型对词元的预测越准确，所需传递的比特数就越少。
    - 因此，平均交叉熵损失也可以理解为模型在压缩文本时每个词元的平均信息量需求。
    - 无论是短文档还是长文档，这种度量都能维持一致的标准。
---

## 小结

* **对隐状态使用循环计算的神经网络称为循环神经网络（RNN）。**
* 循环神经网络的隐状态可以捕获直到当前时间步序列的历史信息。
* 循环神经网络模型的参数数量不会随着时间步的增加而增加。
* 我们可以使用循环神经网络创建字符级语言模型。
* 我们可以使用困惑度来评价语言模型的质量。